In [2]:
from pyExecVaspSlurm import *
import numpy as np
import pandas as pd
import os
from ase.io import read, write

/Users/dominicwelti/opt/anaconda3/lib/python3.9/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
server = Server('st_st179390','justus2.uni-ulm.de', '/lustre/home/st/st_us-031400/st_st179390/') # all refer to server
sftp, ts = server.connect()

Your OTP: Password: <paramiko.Transport at 0x10f37b80 (cipher aes128-ctr, 128 bits) (active; 0 open channel(s))>


In [8]:
# Download h5 files
folders=server.list_folders(sftp,'Graphite_DFT')
results_dir='/Users/dominicwelti/Documents/Master_Thesis_Data_Set/Graphite/DFT'

for folder in folders:
    #name=folder.split("/")[-2]
    sftp.get(f'Graphite_DFT/{folder}/vaspout.h5',f'{results_dir}/{folder}.h5')

In [10]:
# Download POSCAR files for DFT-D3BF
poscar_dir='/Users/dominicwelti/Documents/Master_Thesis_Data_Set/Graphite/POSCAR'
folders_sub=list(filter(lambda x: 'D3BF' in x, folders))

for folder in folders_sub:
    sftp.get(f'Graphite_DFT/{folder}/POSCAR',f'{poscar_dir}/{folder}_POSCAR')

In [58]:
# Graphite: create remote folders for POSCAR files
poscars=list(filter(lambda x:'POSCAR' in x, os.listdir(poscar_dir)))
for poscar in poscars:
    c_par=np.round(float(poscar.split('_')[1]), 2)
    sftp.mkdir(f'Graphite_MLIP/{c_par}')
    sftp.put(f'{poscar_dir}/{poscar}', f'Graphite_MLIP/{c_par}/POSCAR')

In [56]:
# Benzene: create remote folders for ase.xyz files
xyz_folder='/Users/dominicwelti/Documents/Master_Thesis_Data_Set/Benzene/DF-KS_PBE-D3_BJ/aug-cc-pVTZ'
xyz_folders=list(filter(lambda x:'Benzene' in x, os.listdir(xyz_folder)))

for folder in xyz_folders:
    distance=folder.split('_')[-1]
    sftp.mkdir(f'Benzene_MLIP/{distance}')
    sftp.put(f'{xyz_folder}/{folder}/ase.xyz', f'Benzene_MLIP/{distance}/ase.xyz')

In [69]:
sftp.get('Benzene_MLIP/benzene_MACE.pkl', '/Users/dominicwelti/Documents/Master_Thesis_Data_Set/Benzene/MACE/results.pkl')